In [1]:
!pip install transformers
!pip install datasets
!pip install seqeval
!pip install ipywidgets
!pip install IProgress

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import torch

In [3]:
import transformers
from datasets import load_dataset, load_metric

In [4]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [5]:
DATA_FILE = '/notebooks/token_classification/data/pe_dataset_task_first_newer.pt'

In [6]:
dataset = torch.load(DATA_FILE)

AssertionError: 

In [ ]:
dataset

In [199]:
dataset['train'][0]

{'split': 'TRAIN',
 'paragraph': 'When we watch movies or dramas, some of us like to see good people have a good ending and bad people are punished. While there are also some people who do not like this. Personally, I prefer to see good people get rewarded. However, if the good people always get an happy ending, I don’t think it’s proper.',
 'para_tokens_list': ['When',
  'we',
  'watch',
  'movies',
  'or',
  'dramas',
  'some',
  'of',
  'us',
  'like',
  'to',
  'see',
  'good',
  'people',
  'have',
  'a',
  'good',
  'ending',
  'and',
  'bad',
  'people',
  'are',
  'punished',
  'While',
  'there',
  'are',
  'also',
  'some',
  'people',
  'who',
  'do',
  'not',
  'like',
  'this',
  'Personally',
  'I',
  'prefer',
  'to',
  'see',
  'good',
  'people',
  'get',
  'rewarded',
  'However',
  'if',
  'the',
  'good',
  'people',
  'always',
  'get',
  'an',
  'happy',
  'ending',
  'I',
  'don’t',
  'think',
  'it’s',
  'proper'],
 'para_labels_list': ['O',
  'O',
  'O',
  'O',

In [200]:
list_labels = [x for xs in dataset['train']['para_labels_int_list'] for x in xs]

In [201]:
set(list_labels)

{0, 1, 2, 3, 4, 5, 6}

In [202]:
label_list = set(list_labels)

In [225]:
label_list = list(label_list)

In [226]:
label_list

[0, 1, 2, 3, 4, 5, 6]

In [204]:
dataset

DatasetDict({
    train: Dataset({
        features: ['split', 'paragraph', 'para_tokens_list', 'para_labels_list', 'para_labels_int_list', 'is_equal'],
        num_rows: 1082
    })
    test: Dataset({
        features: ['split', 'paragraph', 'para_tokens_list', 'para_labels_list', 'para_labels_int_list', 'is_equal'],
        num_rows: 356
    })
    validation: Dataset({
        features: ['split', 'paragraph', 'para_tokens_list', 'para_labels_list', 'para_labels_int_list', 'is_equal'],
        num_rows: 271
    })
})

In [205]:
from transformers import AutoTokenizer

In [206]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [207]:
label_all_tokens = True

In [131]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["para_tokens_list"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["para_labels_int_list"]):
        #print(i)
        word_ids = tokenized_inputs.word_ids() # this is of length 133
        #print(word_ids)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                # print(word_idx)
                label_ids.append(label)
                # print(word_idx)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels = label_ids
        
#         labels_list = []
        
#         for lst in labels:
#             labels_list.append(lst[2])
    # print(len(labels))
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [132]:
len(dataset['train'][0]['para_tokens_list'])

120

In [133]:
len(dataset['train'][0]['para_labels_int_list'])

120

In [134]:
result = tokenize_and_align_labels(dataset['train'][0])

In [135]:
result

{'input_ids': [101, 2174, 1996, 20502, 2015, 1997, 6681, 2008, 3288, 2000, 2149, 2323, 2025, 2022, 15486, 2011, 2111, 2000, 2707, 2007, 2138, 1997, 4866, 1997, 2478, 6681, 2116, 2111, 3279, 2037, 3105, 2926, 1999, 2312, 2313, 3032, 2152, 12163, 3446, 2089, 3426, 4126, 1998, 2591, 18549, 4237, 2013, 2023, 2478, 1997, 6681, 2024, 2036, 13382, 3809, 10796, 1998, 2943, 4487, 18719, 24952, 2239, 2005, 2742, 13367, 2111, 9544, 2000, 2224, 9841, 1011, 12699, 3698, 2000, 9378, 10447, 1998, 7766, 2738, 2084, 2192, 12699, 2021, 2027, 2106, 2025, 2113, 2008, 4102, 2007, 2192, 12699, 9841, 1011, 12699, 3698, 16678, 2012, 2560, 1016, 2335, 2300, 1998, 6451, 2000, 9378, 2122, 2795, 8059, 2004, 2017, 2064, 5674, 2065, 5190, 1997, 2945, 2224, 9841, 1011, 12699, 6681, 2129, 2116, 2300, 1998, 2129, 2172, 6451, 2097, 2022, 13842, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [136]:
len(result['input_ids'])

133

In [137]:
len(result['attention_mask'])

133

In [138]:
len(result['labels'])

133

In [47]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

IndexError: list index out of range

In [ ]:
# make tags numerical

In [53]:
# try work

In [27]:
example = dataset["train"][0]
print(example["para_tokens_list"])

['However', 'the', 'disadvantages', 'of', 'machines', 'that', 'bring', 'to', 'us', 'should', 'not', 'be', 'neglected', 'by', 'people', 'To', 'start', 'with', 'because', 'of', 'extensive', 'of', 'using', 'machines', 'many', 'people', 'loss', 'their', 'job', 'Especially', 'in', 'large', 'population', 'countries', 'high', 'unemployment', 'rate', 'may', 'cause', 'crime', 'and', 'social', 'instability', 'Apart', 'from', 'this', 'using', 'of', 'machines', 'are', 'also', 'pose', 'serious', 'pollution', 'and', 'energy', 'dissipation', 'For', 'example', 'nowadays', 'people', 'prefer', 'to', 'use', 'dish-washing', 'machine', 'to', 'wash', 'dishes', 'and', 'plates', 'rather', 'than', 'hand', 'washing', 'But', 'they', 'did', 'not', 'know', 'that', 'compared', 'with', 'hand', 'washing', 'dish-washing', 'machine', 'consume', 'at', 'least', '2', 'times', 'water', 'and', 'electricity', 'to', 'wash', 'these', 'tableware', 'As', 'you', 'can', 'imagine', 'if', 'thousands', 'of', 'families', 'use', 'dish-

In [28]:
tokenized_input = tokenizer(example["para_tokens_list"], is_split_into_words=True)

In [29]:
tokenized_input

{'input_ids': [101, 2174, 1996, 20502, 2015, 1997, 6681, 2008, 3288, 2000, 2149, 2323, 2025, 2022, 15486, 2011, 2111, 2000, 2707, 2007, 2138, 1997, 4866, 1997, 2478, 6681, 2116, 2111, 3279, 2037, 3105, 2926, 1999, 2312, 2313, 3032, 2152, 12163, 3446, 2089, 3426, 4126, 1998, 2591, 18549, 4237, 2013, 2023, 2478, 1997, 6681, 2024, 2036, 13382, 3809, 10796, 1998, 2943, 4487, 18719, 24952, 2239, 2005, 2742, 13367, 2111, 9544, 2000, 2224, 9841, 1011, 12699, 3698, 2000, 9378, 10447, 1998, 7766, 2738, 2084, 2192, 12699, 2021, 2027, 2106, 2025, 2113, 2008, 4102, 2007, 2192, 12699, 9841, 1011, 12699, 3698, 16678, 2012, 2560, 1016, 2335, 2300, 1998, 6451, 2000, 9378, 2122, 2795, 8059, 2004, 2017, 2064, 5674, 2065, 5190, 1997, 2945, 2224, 9841, 1011, 12699, 6681, 2129, 2116, 2300, 1998, 2129, 2172, 6451, 2097, 2022, 13842, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [30]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

In [31]:
tokens

['[CLS]',
 'however',
 'the',
 'disadvantage',
 '##s',
 'of',
 'machines',
 'that',
 'bring',
 'to',
 'us',
 'should',
 'not',
 'be',
 'neglected',
 'by',
 'people',
 'to',
 'start',
 'with',
 'because',
 'of',
 'extensive',
 'of',
 'using',
 'machines',
 'many',
 'people',
 'loss',
 'their',
 'job',
 'especially',
 'in',
 'large',
 'population',
 'countries',
 'high',
 'unemployment',
 'rate',
 'may',
 'cause',
 'crime',
 'and',
 'social',
 'instability',
 'apart',
 'from',
 'this',
 'using',
 'of',
 'machines',
 'are',
 'also',
 'pose',
 'serious',
 'pollution',
 'and',
 'energy',
 'di',
 '##ssi',
 '##pati',
 '##on',
 'for',
 'example',
 'nowadays',
 'people',
 'prefer',
 'to',
 'use',
 'dish',
 '-',
 'washing',
 'machine',
 'to',
 'wash',
 'dishes',
 'and',
 'plates',
 'rather',
 'than',
 'hand',
 'washing',
 'but',
 'they',
 'did',
 'not',
 'know',
 'that',
 'compared',
 'with',
 'hand',
 'washing',
 'dish',
 '-',
 'washing',
 'machine',
 'consume',
 'at',
 'least',
 '2',
 'times',

In [118]:
len(tokenized_input.word_ids())

133

In [62]:
tokenized_input.word_ids(batch_index=0)

[None,
 0,
 1,
 2,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 56,
 56,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 64,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 85,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 108,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 None]

In [34]:
word_ids = tokenized_input.word_ids()

In [35]:
word_ids

[None,
 0,
 1,
 2,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 56,
 56,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 64,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 85,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 108,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 None]

In [36]:
aligned_labels = [-100 if i is None else example["para_tokens_list"][i] for i in word_ids]

In [37]:
print(len(aligned_labels), len(tokenized_input["input_ids"]))

133 133


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["para_tokens_list"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["para_labels_list"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                # print(word_idx)
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [47]:
for i, label in enumerate(example["para_labels_int_list"]):
    print(i, label)

0 0
1 0
2 0
3 0
4 0
5 1
6 2
7 2
8 2
9 2
10 2
11 2
12 2
13 2
14 2
15 2
16 2
17 2
18 2
19 2
20 2
21 2
22 2
23 0
24 0
25 0
26 0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0


### own tokenize and align fx

In [208]:
def tokenize_and_align_labels_own(examples):
    tokenized_inputs = tokenizer(examples["para_tokens_list"], truncation=True, is_split_into_words=True)
    #print(examples['paragraph'])
    word_ids = tokenized_inputs.word_ids()
    
    labels_list = []
    
    for word_idx in word_ids:
    
        previous_idx = None

        if word_idx == None:

            labels_list.append(-100)

        elif word_idx == previous_idx:

            labels_list.append(examples["para_labels_int_list"][previous_idx])

        else:
            # print(word_idx) [None, 119, 56, 56, 124, None]
            labels_list.append(examples["para_labels_int_list"][word_idx])

        previous_idx = word_idx
        
        
    tokenized_inputs["labels"] = labels_list
    return tokenized_inputs

In [210]:
tokenize_and_align_labels_own(dataset['train'][0])

{'input_ids': [101, 2043, 2057, 3422, 5691, 2030, 16547, 2070, 1997, 2149, 2066, 2000, 2156, 2204, 2111, 2031, 1037, 2204, 4566, 1998, 2919, 2111, 2024, 14248, 2096, 2045, 2024, 2036, 2070, 2111, 2040, 2079, 2025, 2066, 2023, 7714, 1045, 9544, 2000, 2156, 2204, 2111, 2131, 14610, 2174, 2065, 1996, 2204, 2111, 2467, 2131, 2019, 3407, 4566, 1045, 2123, 1521, 1056, 2228, 2009, 1521, 1055, 5372, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100]}

In [212]:
tokenized_datasets = dataset.map(tokenize_and_align_labels_own, batched=False)

  0%|          | 0/1082 [00:00<?, ?ex/s]

  0%|          | 0/356 [00:00<?, ?ex/s]

  0%|          | 0/271 [00:00<?, ?ex/s]

## Model

In [227]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embd

In [228]:
model

DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
          

In [229]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
          

In [230]:
device

device(type='cuda')

In [231]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    #push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [232]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [233]:
metric = load_metric("seqeval")

In [234]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [235]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [236]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: para_labels_list, para_tokens_list, para_labels_int_list, is_equal, paragraph, split.
***** Running training *****
  Num examples = 1082
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 204


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.694735,0.000000,0.000000,0.000000,0.789526
2,No log,0.630983,0.000000,0.000000,0.000000,0.799482
3,No log,0.617852,0.000000,0.000000,0.000000,0.802220


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: para_labels_list, para_tokens_list, para_labels_int_list, is_equal, paragraph, split.
***** Running Evaluation *****
  Num examples = 271
  Batch size = 16
/opt/conda/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 3 seems not to be NE tag.
  warnings.warn('{} seems not to b

TrainOutput(global_step=204, training_loss=0.7183196123908547, metrics={'train_runtime': 29.7419, 'train_samples_per_second': 109.139, 'train_steps_per_second': 6.859, 'total_flos': 109240460013300.0, 'train_loss': 0.7183196123908547, 'epoch': 3.0})

In [237]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: para_labels_list, para_tokens_list, para_labels_int_list, is_equal, paragraph, split.
***** Running Evaluation *****
  Num examples = 271
  Batch size = 16


{'eval_loss': 0.6178519129753113,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_f1': 0.0,
 'eval_accuracy': 0.8022202309836718,
 'eval_runtime': 1.2844,
 'eval_samples_per_second': 210.999,
 'eval_steps_per_second': 13.236,
 'epoch': 3.0}

In [238]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: para_labels_list, para_tokens_list, para_labels_int_list, is_equal, paragraph, split.
***** Running Prediction *****
  Num examples = 271
  Batch size = 16


{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 0.8022202309836718}

In [139]:
# new try

In [141]:
examples = dataset['train'][0]

In [142]:
tokenized_inputs = tokenizer(examples["para_tokens_list"], truncation=True, is_split_into_words=True)

In [143]:
tokenized_inputs

{'input_ids': [101, 2174, 1996, 20502, 2015, 1997, 6681, 2008, 3288, 2000, 2149, 2323, 2025, 2022, 15486, 2011, 2111, 2000, 2707, 2007, 2138, 1997, 4866, 1997, 2478, 6681, 2116, 2111, 3279, 2037, 3105, 2926, 1999, 2312, 2313, 3032, 2152, 12163, 3446, 2089, 3426, 4126, 1998, 2591, 18549, 4237, 2013, 2023, 2478, 1997, 6681, 2024, 2036, 13382, 3809, 10796, 1998, 2943, 4487, 18719, 24952, 2239, 2005, 2742, 13367, 2111, 9544, 2000, 2224, 9841, 1011, 12699, 3698, 2000, 9378, 10447, 1998, 7766, 2738, 2084, 2192, 12699, 2021, 2027, 2106, 2025, 2113, 2008, 4102, 2007, 2192, 12699, 9841, 1011, 12699, 3698, 16678, 2012, 2560, 1016, 2335, 2300, 1998, 6451, 2000, 9378, 2122, 2795, 8059, 2004, 2017, 2064, 5674, 2065, 5190, 1997, 2945, 2224, 9841, 1011, 12699, 6681, 2129, 2116, 2300, 1998, 2129, 2172, 6451, 2097, 2022, 13842, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [145]:
len(tokenized_inputs['input_ids'])

133

In [146]:
word_ids = tokenized_inputs.word_ids()

In [147]:
word_ids

[None,
 0,
 1,
 2,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 56,
 56,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 64,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 85,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 108,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 None]

In [148]:
len(word_ids)

133

In [160]:
labels_list = []

In [161]:
for word_idx in word_ids:
    
    previous_idx = None
    
    if word_idx == None:
        
        labels_list.append(-100)
        
    elif word_idx == previous_idx:
        
        labels_list.append(examples["para_labels_int_list"][previous_idx])
        
    else:
        
        labels_list.append(examples["para_labels_int_list"][word_idx])
        
    previous_idx = word_idx

In [162]:
len(labels_list)

133

In [168]:
labels_list

[-100,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -100]

In [165]:
new_list = []

In [166]:
for word_idx in word_ids:
    
    previous_idx = None
    
    if word_idx == None:
        
        new_list.append(-100)
        
    elif word_idx == previous_idx:
        
        new_list.append(examples["para_tokens_list"][previous_idx])
        
    else:
        
        new_list.append(examples["para_tokens_list"][word_idx])
        
    previous_idx = word_idx

In [167]:
new_list

[-100,
 'However',
 'the',
 'disadvantages',
 'disadvantages',
 'of',
 'machines',
 'that',
 'bring',
 'to',
 'us',
 'should',
 'not',
 'be',
 'neglected',
 'by',
 'people',
 'To',
 'start',
 'with',
 'because',
 'of',
 'extensive',
 'of',
 'using',
 'machines',
 'many',
 'people',
 'loss',
 'their',
 'job',
 'Especially',
 'in',
 'large',
 'population',
 'countries',
 'high',
 'unemployment',
 'rate',
 'may',
 'cause',
 'crime',
 'and',
 'social',
 'instability',
 'Apart',
 'from',
 'this',
 'using',
 'of',
 'machines',
 'are',
 'also',
 'pose',
 'serious',
 'pollution',
 'and',
 'energy',
 'dissipation',
 'dissipation',
 'dissipation',
 'dissipation',
 'For',
 'example',
 'nowadays',
 'people',
 'prefer',
 'to',
 'use',
 'dish-washing',
 'dish-washing',
 'dish-washing',
 'machine',
 'to',
 'wash',
 'dishes',
 'and',
 'plates',
 'rather',
 'than',
 'hand',
 'washing',
 'But',
 'they',
 'did',
 'not',
 'know',
 'that',
 'compared',
 'with',
 'hand',
 'washing',
 'dish-washing',
 'dish-

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv('/notebooks/token_classification/data/PE_dataset_pour_token_classification_task.csv')